In [ ]:
import pandas as pd
import numpy as np
import requests
import re
import urllib.parse
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from pathlib import Path

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

---

## Official Joke API

Generates random jokes (from jokes submitted to API)

[Documentation](https://github.com/15Dkatz/official_joke_api)


In [ ]:
##baseurl
base_url = "https://official-joke-api.appspot.com/"


#### 10 random jokes

In [ ]:

# endpoint for 10 random jokes
endpoint = 'random_ten'
url = base_url + endpoint

In [ ]:
## What does it look like if you visit this url?
url

In [ ]:
r = requests.get(url)

In [ ]:
r.status_code

In [ ]:
r.json()

In [ ]:
pd.json_normalize(r.json())

#### Joke by id

In [ ]:
## endpoint for joke by id

endpoint = 'jokes/1'
r = requests.get(base_url+endpoint)
r.ok

In [ ]:
r.status_code

In [ ]:
r.json()

---

## Zippopotam API

Postal code lookup.

Endpoints:
* Places by postal code `/{country}/{postalcode}`
* Postal codes by place `/{country}/{state}/{place}`
* Places near postal code `/nearby/{country}/{postalcode}`

[Documentation](https://docs.zippopotam.us/docs/getting-started/) and also [this page](https://www.zippopotam.us/)

In [ ]:
base_url = 'https://api.zippopotam.us'

#### places by postal code

In [ ]:
# places by postal code
options = ["us", "43420"] # ["county", "postalcode"]
url = f"{base_url}/{'/'.join(options)}"
url


In [ ]:
newurl = base_url + "/us/84602"
newurl

In [ ]:
r = requests.get(url)
r.ok

In [ ]:
r.json()

#### Postal codes by place

In [ ]:
options = ["us", "tx", "houston"]

## This line will ensure that the url is formed correctly (for example if there are spaces)
encoded_options = [urllib.parse.quote(option) for option in options]
url = f"{base_url}/{'/'.join(encoded_options)}"
r = requests.get(url)
r.ok


In [ ]:
url

In [ ]:
r.json()

In [ ]:
r.json().keys()

In [ ]:
pd.json_normalize(r.json())

In [ ]:
df = pd.json_normalize(r.json()['places'])
df['longitude'] = df['longitude'].astype(float)
df['latitude'] = df['latitude'].astype(float)


---

## Free Geocoding API

**An API Key is now required**

[Documentation](https://geocode.maps.co/)

* The endpoints show that the query parameters should be entered as url parameters (indicated by the `?` in the url)
* We can easily specify the query options using the `params` argument in the `.get` method
* In order to do this, set up a dictionary such that 
    * the key is the parameter name (indicated in the documentation)
    * the value is the our query choice
* `requests.get()` will automatically make sure that our parameters are formatted into valid urls

Endpoints:
* Forward geocode (`https://geocode.maps.co/search?q={address}`)

In [ ]:
base_url = "https://geocode.maps.co"
forward = '/search'

params = {'q':'801 Smith Road Fremont OH 43420'}

r = requests.get(base_url + forward, params = params)
print(r.status_code)
print(r.url)

In [ ]:
r.json()

Endpoints
* Reverse geocode (`https://geocode.maps.co/reverse?lat={latitude}&lon={longitude}`)

In [ ]:
reverse = '/reverse'

params = {'lat': '41.299294', 'lon': '-83.091367'}

r = requests.get(base_url + reverse, params = params)
print(r.status_code)
print(r.url)

In [ ]:
r.json()

---

## NASA APOD API

[NASA APIs](https://api.nasa.gov/).  
* Sign up for an API key by filling out the form
* Check your email for the API key
* Create a .txt file and save *just the text of the api key* in the file
    * For example, in the code below, my API key is saved in a file called "`nasa_api_key.txt`"

#### APOD Query Parameters

| Parameter   | Type        | Default   | Description                                                                                                      |
|-------------|-------------|-----------|------------------------------------------------------------------------------------------------------------------|
| date        | YYYY-MM-DD  | today     | The date of the APOD image to retrieve                                                                           |
| start_date  | YYYY-MM-DD  | none      | The start of a date range, when requesting date for a range of dates. Cannot be used with date.                   |
| end_date    | YYYY-MM-DD  | today     | The end of the date range, when used with start_date.                                                            |
| count       | int         | none      | If this is specified then count randomly chosen images will be returned. Cannot be used with date or start_date and end_date. |
| thumbs      | bool        | False     | Return the URL of video thumbnail. If an APOD is not a video, this parameter is ignored.                          |
| api_key     | string      | DEMO_KEY  | api.nasa.gov key for expanded usage                                                                              |

Example query:
```
https://api.nasa.gov/planetary/apod?api_key=DEMO_KEY
```

In [ ]:
base_url = "https://api.nasa.gov/planetary/apod"

In [ ]:
# # set the text of my api key to a variable called `nasa_key`
# # Use this code if the file "nasa_api_key.txt" is in the same directory as this script
# with open('~/Documents/Work/apiKeys/nasa_api_key.txt', 'r') as file:
#     nasa_key = file.read()

# # This is code that I use to read my api key from a file in a different directory on my computer
# # Students can ignore this code
file_path = Path.home() / "Documents" / "Work" / "apiKeys" / "nasa_api_key.txt"

if file_path.exists():
    with file_path.open("r") as file:
        nasa_key = file.read().strip()
else:
    print("API key file not found!")

Let's query all the APODs from Oct 1, 2023 - Oct 7, 2023
* The documentation specifies that query parameters should be entered as url parameters (indicated by the `?` in the url)
* We can easily specify the query options using the `params` argument in the `.get` method
* In order to do this, set up a dictionary such that 
    * the key is the parameter name (indicated in the documentation)
    * the value is the our query choice
* `requests.get()` will automatically make sure that our parameters are formatted into valid urls


In [ ]:
params = {'api_key':nasa_key,
          'start_date': '2023-10-01',
          'end_date': '2023-10-07'}

In [ ]:
r = requests.get(base_url, params=params)

In [ ]:
#r.url

In [ ]:
r.status_code

In [ ]:
r.json()

In [ ]:
pd.json_normalize(r.json())

---

## Exchange Rates API

[Documentation](https://apilayer.com/marketplace/exchangerates_data-api)

Sign up for an API key:
* Go to [https://apilayer.com/marketplace/exchangerates_data-api](https://apilayer.com/marketplace/exchangerates_data-api)
* You'll have to verify your email address and sign in
* If you are taken back to the main page, search for "exchange rates" and click on "Exchange Rates Data API"
* Under Pricing, find "Free Plan" and click "Subscribe"
* If everything went well, you should then see a screen that says "Subscription successful" and "Your API Key"
* Copy your API key to the clipboard and then save it in a .txt file (choose a sensible name for the file like "`exchange_rates_apikey.txt`")
* **NEVER NEVER NEVER post your API key in a public place.  Anytime you use Git or GitHub with any API key, include the file with the key information in the `.gitignore` file**

### Authenticating 

The NASA API, wanted us to authenticate with the api-key as a parameter in the url.  However, the documentation for this API says that the API key should be specified in a **header**.
* Headers provide metadata about the request or response.  Some APIs (such as this one) require that the API key be specified in a header. 
    * Headers might also contain content type, return data type, user-agent, etc.  Generally the API documentation will specify 


### Endpoints
* `/convert`
* `/fluctuation`
* `/latest`
* `/symbols`
* `/timeseries`
* `/{date}`

Each endpoint has its own parameters (or sometimes no parameters).  As with many API endpoints, some parameters are required and others are optional.


#### Example
For example, let's look at the `/convert` endpoint.  There are three required parameters and one optional:
* amount (required) - amount to be converted
* from (required) - three-letter currency code for the currency you would like to convert from
* to (required) - three-letter currency code for the currency you would like to convert to
* date (optional) - specify a date to use historical rates (default is latest)


In [ ]:
## First, get the api key
# with open('exchange_rates_apikey.txt', 'r') as file:
#     apikey = file.read()

file_path = Path.home() / "Documents" / "Work" / "apiKeys" / "exchange_rates_apikey.txt"
if file_path.exists():
    with file_path.open("r") as file:
        apikey = file.read().strip()
else:
    print("API key file not found!")

In [ ]:
# set up the headers
headers= {
  "apikey": apikey
}

In [ ]:
# specify base_url and endpoint url (or this could be done in one step)
base_url = "https://api.apilayer.com/exchangerates_data"
endpoint = '/convert'
url = base_url + endpoint

In [ ]:
# set up parameters.  I'm going to specify 35 Euros to US Dollars for Oct 8, 2023
params = {'amount':'35',
          'from':'EUR',
          'to':'USD',
          'date': '2024-10-10'}

In [ ]:
r = requests.get(url, headers=headers, params=params)
print(r.ok)
print(r.url)

In [ ]:
r.json()

## Wikipedia API

[API Documentation](https://www.mediawiki.org/wiki/API:Main_page)

Some APIs (and websites) want you to specify who you are through a `user-agent` parameter.  This can be the browser user-agent, but it can also be just a description of your project.  

The Wikipedia API requests that you specify the user-agent per their [policy](https://meta.wikimedia.org/wiki/User-Agent_policy).  The basic format is `Project name (email@email.com)`

In [ ]:
# user_agent = 'Practice with APIs for Stat 386/0.0 (esnt@byu.edu)'
# headers = {'User-Agent':user_agent}

In [ ]:
base_url = "https://en.wikipedia.org/w/api.php"
params = {'action':'query',
          'list':'search',
          'srsearch': 'Star Wars',
          'format': 'json'}

In [ ]:
r = requests.get(base_url, params=params)

In [ ]:
data = r.json()
print(len(data['query']['search']))
for result in data['query']['search']:
    print(f"Title: {result['title']}\nSnippet: {result['snippet']}\n\n")
    

---

## The Movie Database (TMDb)

[Website](https://www.themoviedb.org/?language=en-US) and [API Documentation](https://developer.themoviedb.org/docs)

In [ ]:
base_url = "https://api.themoviedb.org/3/search/movie"

with open('tmdb_api_key.txt', 'r') as file:
    apikey = file.readlines()[1].strip()


In [ ]:
headers = {
    "accept": "application/json",
    "Authorization": "Bearer " + apikey}

In [ ]:
results = []
for page in range(1,10,1):

    params = {'include_adult':'false',
            'language':"en-US",
            'query':'Star Wars',
            'page': str(page)}
    
    response = requests.get(base_url, headers=headers, params=params)

    print(response.status_code)
    if response.ok:
        results.extend(response.json()['results'])

In [ ]:
sw = pd.DataFrame(results)
sw = sw[['original_title','popularity','release_date','title','vote_average','vote_count']]

In [ ]:
sw.sort_values(by='popularity', ascending=False).head(10)

In [ ]:
sw[sw.vote_count>500].sort_values(by='vote_average', ascending=False).head(10)